In [ ]:
#!conda update conda
#!pip install update pip
#!pip install update cobra
#!pip install update pip
#!pip install update  joblib 
#!pip install update  multiprocess
#!pip install update networkx
#!pip install update  pandas 
#!pip install update numpy 
#!pip install update scipy    
#normal.reactions.get_by_id("PHETHPTOX2").bounds = (0, 0.0)
#normal.reactions.get_by_id("r0399").bounds = (0.0, 0.0)
#cobra.io.save_json_model(normal, "PKU.json")
#model = cobra.io.load_matlab_model('GEM_Recon3_thermocurated_redHUMAN.mat')
#cobra.io.save_json_model(model, "GEM_Recon3_thermocurated_redHUMAN.json")

In [4]:
import cobra
import numpy as np
import warnings 
warnings.filterwarnings("ignore")

#GEM_Recon2_thermocurated_redHUMAN = cobra.io.load_matlab_model('GEM_Recon2_thermocurated_redHUMAN.mat')
#cobra.io.save_json_model(GEM_Recon2_thermocurated_redHUMAN, "GEM_Recon2_thermocurated_redHUMAN.json")
normal = cobra.io.load_json_model("GEM_Recon2_thermocurated_redHUMAN.json")
normal

Name,model
Memory address,0x07f14b058d700
Number of metabolites,5063
Number of reactions,7440
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"c, l, m, r, e, x, n, g"


In [5]:
print(
normal.reactions.get_by_id("PHETHPTOX2").bounds,
normal.reactions.get_by_id("r0399").bounds)

(0.0, 100.0) (-100.0, 100.0)


In [8]:
import pandas as pd

#por ejemplo una funcion: input: lista de metabolitos; output: todos los campos.
frozen       = normal.metabolites.get_by_id("tyr_L_e").reactions.union(normal.metabolites.get_by_id("tyr_L_c").reactions).union(normal.metabolites.get_by_id("tyr_L_m").reactions)
tyr_L_ids    = [list(frozen)[i].id for i in range(0,len(frozen))]
tyr_L_names  = [list(frozen)[i].name for i in range(0,len(frozen))]
tyr_L_rxns   = [list(frozen)[i].reaction for i in range(0,len(frozen))]
list_of_tuples   = list(zip(tyr_L_ids, tyr_L_names, tyr_L_rxns))  
columns          = ["tyr_L_e_ids","tyr_L_e_names","tyr_L_e_rxns"]
tyr_L          =  pd.DataFrame(list_of_tuples, columns = columns)  

frozen         = normal.metabolites.get_by_id("phe_L_e").reactions.union(normal.metabolites.get_by_id("phe_L_c").reactions).union(normal.metabolites.get_by_id("phe_L_m").reactions)
phe_L_ids    = [list(frozen)[i].id for i in range(0,len(frozen))]
phe_L_names  = [list(frozen)[i].name for i in range(0,len(frozen))]
phe_L_rxns   = [list(frozen)[i].reaction for i in range(0,len(frozen))]

list_of_tuples   = list(zip(phe_L_ids,phe_L_names, phe_L_rxns))  
columns          = ["phe_L_e_ids","phe_L_e_names","phe_L_e_rxns"]
phe_L          =  pd.DataFrame(list_of_tuples, columns = columns)

## FBA

In [78]:

solution_normal = normal.optimize()
phe_L_fluxes   = solution_normal.fluxes[phe_L_ids].to_frame() #
phe_L_fluxes.query('abs(fluxes) > 1e-10', inplace=True)
phe_L_fluxes.rename_axis('phe_L_e_ids').reset_index()
phe_L_fluxes = phe_L_fluxes.rename_axis('phe_L_e_ids').reset_index()
pd.merge(phe_L_fluxes, phe_L)

,phe_L_e_ids,fluxes,phe_L_e_names,phe_L_e_rxns
0,r1764,0.829787,Y+LAT2 Utilized transport,gln_L_c + na1_e + phe_L_e --> gln_L_e + na1_c ...
1,EX_phe_L_e,-0.829787,exchange reaction for L-phenylalanine,phe_L_e <--
2,biomass,3.198056,Generic human biomass reaction,0.505626 ala_L_c + 0.35926 arg_L_c + 0.279425 ...


In [77]:
tyr_L_fluxes   = sol_normal.fluxes[tyr_L_ids].to_frame()  #.query('abs(fluxes) > 1e-7', inplace=True)
tyr_L_fluxes.query('abs(fluxes) > 1e-10', inplace=True)
tyr_L_fluxes = tyr_L_fluxes.rename_axis('tyr_L_e_ids').reset_index()
pd.merge(tyr_L_fluxes, tyr_L)

,tyr_L_e_ids,fluxes,tyr_L_e_names,tyr_L_e_rxns
0,EX_tyr_L_e,-0.510637,L-Tyrosine exchange,tyr_L_e <=>
1,r1634,-1.744680,Amino Acid-Polyamine-Organocation (APC) TCDB:2...,ile_L_c + tyr_L_e <=> ile_L_e + tyr_L_c
2,TYRt,2.255317,L-tyrosine transport,tyr_L_e <=> tyr_L_c
3,biomass,3.198056,Generic human biomass reaction,0.505626 ala_L_c + 0.35926 arg_L_c + 0.279425 ...


# Phenotypic space

## Finding glucose and oxygen uptakes

In [110]:
normal.metabolites[1]

Metabolite identifier,10fthf5glu_l
Name,10-formyltetrahydrofolate-[Glu](5)
Memory address,0x07f14961c84f0
Formula,C40H45N11O19
Compartment,l
In 3 reaction(s),"GGH_10FTHF6GLUl, 10FTHF5GLUtl, GGH_10FTHF5GLUl"


In [127]:
#normal.
#[list(frozen)[i].reaction for i in range(0,len(frozen))]

rxn_names      = [normal.reactions[i].name for i in range(0, len(normal.reactions))]
rxn_ids        = [normal.reactions[i].id for i in range(0, len(normal.reactions))]
rxn_reaction   = [normal.reactions[i].reaction for i in range(0, len(normal.reactions))]

model_df_rxns       = pd.DataFrame(list(zip(rxn_ids,rxn_names, rxn_reaction)), columns = ['rxn_ids','rxn_names',"rxn_reaction"])


met_names      = [normal.metabolites[i].name for i in range(0, len(normal.metabolites))]
met_ids        = [normal.metabolites[i].id for i in range(0, len(normal.metabolites))]
met_formula   = [normal.metabolites[i].formula for i in range(0, len(normal.metabolites))]

model_df_mets       = pd.DataFrame(list(zip(met_ids,met_names, met_formula)), columns = ['met_ids','met_names',"met_formula"])

model_df_mets[model_df_mets.met_ids.str.contains('glc_D_')]
#model_df_mets[model_df_mets.met_ids.str.contains('^o2_')]

,met_ids,met_names,met_formula
628,glc_D_e,D-glucose,C6H12O6
810,glc_D_c,D-glucose,C6H12O6
907,glc_D_r,D-glucose,C6H12O6
1944,glc_D_l,D-glucose,C6H12O6
2032,glc_D_g,D-glucose,C6H12O6


In [126]:
def metabolites2reactions(modelo, metabolitos): # Toma metabolitos
  """Crea un agregado de las reacciones segun un listado de metabolitos. 

  Parameters
  ----------
  modelo: model
      El modelo inicial del cual se toman las reacciones
  metabolitos : list
      IDs de un metabolito. La función acepta multiples para crear un joint. 

  Returns
  -------
  df
      crea un dataframe pandas agregada de ids, nombre de reacciones, y reacciones en si
  """
  frozen = modelo.metabolites.get_by_id(metabolitos[0]).reactions # Crea el inicio
  for met in metabolitos: # Añade metabolistos al listado de reacciones
    frozen = frozen.union(modelo.metabolites.get_by_id(met).reactions)
  
  ids   = [list(frozen)[i].id       for i in range(0,len(frozen))] # List Comprehesion
  names = [list(frozen)[i].name     for i in range(0,len(frozen))]
  rxns  = [list(frozen)[i].reaction for i in range(0,len(frozen))]

  list_of_tuples = list(zip(ids, names, rxns))     # DataFrame en Python; crea triplets
  columns        = ["ids","names","rxns"]   # Titulos de columnas de la tripleta
  df             =  pd.DataFrame(list_of_tuples, columns = columns) # Titulos asignados a la lista de tuplas
  
  return df

In [136]:
glucose_oxygen = metabolites2reactions(normal, ["o2_e", "glc_D_e"])
glucose_oxygen[glucose_oxygen.names.str.contains('exchange')]

,ids,names,rxns
3,EX_o2_e,exchange reaction for oxugen,o2_e <--
26,EX_glc_e,D-Glucose exchange,glc_D_e <=>


## Varying glucose and oxygen uptakes

### Uptakes bounds in the model and in the optimal solution

In [246]:
normal.reactions.get_by_id("EX_o2_e").reaction
#normal.reactions.get_by_id("EX_glc_e").reaction

'o2_e <-- '

In [253]:
upkates = ["EX_o2_e","EX_glc_e"]
solution_normal[upkates]

EX_o2_e    -6.933845
EX_glc_e    0.000000
Name: fluxes, dtype: float64

In [252]:
pd.set_option('display.max_rows', None)

solution_normal_df          = pd.merge(solution_normal.to_frame().rename_axis('rxn_ids').reset_index(),model_df_rxns)
solution_normal_df_filtered = solution_normal_df.query('abs(fluxes) > 1e-10') 
df_EXs                      = solution_normal_df_filtered[solution_normal_df_filtered.rxn_ids.str.contains('EX')]
df_EXs.query('(fluxes) < 0') 

,rxn_ids,fluxes,reduced_costs,rxn_names,rxn_reaction
1214,EX_4abut_e,-0.531917,0.000000e+00,4-Aminobutanoate exchange,4abut_e <=>
1245,EX_ach_e,-1.000000,4.146097e-16,Acetylcholine exchange,ach_e <=>
1263,EX_ala_D_e,-1.000000,-5.133582e-16,D-Alanine exchange,ala_D_e <=>
1277,EX_arg_L_e,-1.000000,-1.316672e-14,L-Arginine exchange,arg_L_e <=>
1279,EX_asn_L_e,-1.000000,-2.122567e-15,exchange reaction for L-asparagine,asn_L_e <=>
1281,EX_asp_L_e,-1.000000,-1.316198e-15,L-Aspartate exchange,asp_L_e <=>
1289,EX_biocyt_e,-0.893613,0.000000e+00,Biocytin exchange,biocyt_e <=>
1300,EX_chol_e,-0.547688,6.743931e-17,exchange reaction for Choline,chol_e <--
1302,EX_chsterol_e,-0.065244,0.000000e+00,exchange reaction for cholesterol,chsterol_e <=>
1311,EX_creat_e,-0.148933,0.000000e+00,Creatine exchange,creat_e <=>


## Make the model glucose- and oxygen dependent

In [254]:
solution_normal2.shadow_prices.to_frame()

,shadow_prices
10fthf5glu_c,1.072711e-15
10fthf5glu_l,1.072711e-15
10fthf5glu_m,-6.450999e-17
10fthf6glu_c,1.320347e-15
10fthf6glu_l,1.320347e-15
10fthf6glu_m,0.000000e+00
10fthf7glu_c,1.567982e-15
10fthf7glu_l,1.567982e-15
10fthf7glu_m,0.000000e+00
10fthf_c,8.216670e-17


In [276]:
model_df_mets[model_df_mets.met_names.str.contains('bio')]

,met_ids,met_names,met_formula
477,ksi_pre18_g,"keratan sulfate I biosynthesis, precursor 18",C163H262N11O132S5X
478,ksi_pre19_g,"keratan sulfate I biosynthesis, precursor 19",C171H275N12O137S5X
479,ksi_pre21_g,"keratan sulfate I biosynthesis, precursor 21",C177H284N12O145S6X
480,ksi_pre22_g,"keratan sulfate I biosynthesis, precursor 22",C185H297N13O150S6X
481,ksi_pre24_g,"keratan sulfate I biosynthesis, precursor 24",C191H306N13O158S7X
482,ksi_pre25_g,"keratan sulfate I biosynthesis, precursor 25",C199H319N14O163S7X
483,ksi_pre27_g,"keratan sulfate I biosynthesis, precursor 27",C205H328N14O171S8X
484,ksi_pre28_g,"keratan sulfate I biosynthesis, precursor 28",C213H341N15O176S8X
485,ksi_pre30_g,"keratan sulfate I biosynthesis, precursor 30",C219H350N15O184S9X
486,ksi_pre31_g,"keratan sulfate I biosynthesis, precursor 31",C227H363N16O189S9X


## Parsimonious FBA

pfba_solution = cobra.flux_analysis.pfba(normal)
sol_normal = pfba_solution

phe_L_fluxes   = sol_normal.fluxes[phe_L_ids].to_frame() #.query('abs(fluxes) > 1e-7', inplace=True)
phe_L_fluxes.query('abs(fluxes) > 1e-7', inplace=True)
phe_L_fluxes = phe_L_fluxes.rename_axis('phe_L_e_ids').reset_index()
pd.merge(phe_L_fluxes, phe_L)

tyr_L_fluxes   = sol_normal.fluxes[tyr_L_ids].to_frame()  #.query('abs(fluxes) > 1e-7', inplace=True)
tyr_L_fluxes.query('abs(fluxes) > 1e-7', inplace=True)
tyr_L_fluxes = tyr_L_fluxes.rename_axis('tyr_L_e_ids').reset_index()
pd.merge(tyr_L_fluxes, tyr_L)

## geometric FBA

geometric_fba_sol = cobra.flux_analysis.geometric_fba(normal, epsilon=1e-01, max_tries=50)
sol_normal = geometric_fba_sol
phe_L_fluxes   = sol_normal.fluxes[phe_L_e_ids].to_frame() #.query('abs(fluxes) > 1e-7', inplace=True)
phe_L_fluxes.query('abs(fluxes) > 1e-7', inplace=True)
phe_L_fluxes = phe_L_fluxes.rename_axis('phe_L_e_ids').reset_index()
pd.merge(phe_L_fluxes, phe_L)